In [1]:
%matplotlib inline
from ftplib import FTP
import tempfile
import pandas as pd
import numpy as np
import re  # regex searching
import matplotlib.pyplot as plt

In [2]:
ftp = FTP('ftp.ncdc.noaa.gov')
ftp.login()
ftp.cwd('/pub/data/normals/1981-2010/products/precipitation/')

'250 CWD command successful'

In [3]:
DAILY_FILE_NAMING = 'dly-*-pctall-*.txt'
DAILY_COLUMNS = [('%d' % i) for i in xrange(1,32)]
DAILY_COLUMNS.insert(0, 'MONTH')
DAILY_COLUMNS.insert(0, 'STNID')

In [4]:
NON_NUMERIC_REGEX = re.compile(r'[^\d.]+')
SPECIAL_VALUES = ['-7777', '-8888', '-9999', '-6666', '-5555']  # columns that represent a specific type of NULL data
def process_value_column(string):
    if string in SPECIAL_VALUES:
        return np.nan
    deg = NON_NUMERIC_REGEX.sub('', string)
    deg = float(deg[-3:-1] + '.' + deg[-1])        
    return float(deg)  # return Flag as well.

In [5]:
def process_dataframe(df, value):
    """Process dataframe, setting indexes, etc."""
    value = value.upper()
    df = pd.melt(df, id_vars=['STNID', 'MONTH'], var_name='DAY', value_name=value)
    df[value + '_FLAG'] = df[value].map(lambda s: s[-1])
    df[value] = df[value].apply(process_value_column)
    df['DAY'] = df['DAY'].apply(int)
    df['MONTH'] = df['MONTH'].apply(int)
    df = df.sort_values(['STNID','MONTH', 'DAY'])
    df = df.set_index(['STNID', 'MONTH', 'DAY'])
    return df

In [6]:
def parse_file(filename, columns):
    """Parse a daily weather file."""
    tempf = tempfile.mktemp()
    ftp.retrbinary('RETR %s' % filename, open(tempf, 'wb').write)
    df = pd.read_fwf(open(tempf, 'r'), header=None, names=columns, index_col=None)
    value = re.search(r'dly-(.*)-(.*).txt', filename)
    value = value.group(1) + '_' + value.group(2)
    return process_dataframe(df, value)

In [7]:
# Downloading list of stations
# df = pd.DataFrame(columns=DAILY_COLUMNS)
dfs = []
for filename in ftp.nlst(DAILY_FILE_NAMING):
    filetype = re.search(r'dly-(.*)-(.*).txt', filename).group(1)
    print('Filetype: %s' % filetype)
    df = parse_file(filename, DAILY_COLUMNS)
    dfs.append(df)

Filetype: snwd-pctall
Filetype: snwd-pctall
Filetype: snwd-pctall
Filetype: snwd-pctall
Filetype: snow-pctall
Filetype: snow-pctall
Filetype: snow-pctall
Filetype: snow-pctall
Filetype: snow-pctall
Filetype: prcp-pctall
Filetype: prcp-pctall
Filetype: prcp-pctall
Filetype: prcp-pctall


In [8]:
# Combines list of DataFrames into one
data = pd.concat(dfs, axis=1)

In [9]:
data.head()

SNWD-PCTALL_GE010WI SNWD-PCTALL_GE010WI_FLAG  \
STNID       MONTH DAY                                                 
AQC00914000 1     1                    0.0                        P   
                  2                    0.0                        P   
                  3                    0.0                        P   
                  4                    0.0                        P   
                  5                    0.0                        P   

                       SNWD-PCTALL_GE005WI SNWD-PCTALL_GE005WI_FLAG  \
STNID       MONTH DAY                                                 
AQC00914000 1     1                    0.0                        P   
                  2                    0.0                        P   
                  3                    0.0                        P   
                  4                    0.0                        P   
                  5                    0.0                        P   

                       SNWD-PCTALL_GE003WI SNWD-PCTALL_GE003WI_FLAG  \
STNID       MONTH DAY                                                 
AQC00914000 1     1                    0.0                        P   
                  2                    0.0                        P   
                  3                    0.0                        P   
                  4                    0.0                        P   
                  5                    0.0                        P   

                       SNWD-PCTALL_GE001WI SNWD-PCTALL_GE001WI_FLAG  \
STNID       MONTH DAY                                                 
AQC00914000 1     1                    0.0                        P   
                  2                    0.0                        P   
                  3                    0.0                        P   
                  4                    0.0                        P   
                  5                    0.0                        P   

                       SNOW-PCTALL_GE100TI SNOW-PCTALL_GE100TI_FLAG  \
STNID       MONTH DAY                                                 
AQC00914000 1     1                    0.0                        P   
                  2                    0.0                        P   
                  3                    0.0                        P   
                  4                    0.0                        P   
                  5                    0.0                        P   

                                ...             SNOW-PCTALL_GE001TI  \
STNID       MONTH DAY           ...                                   
AQC00914000 1     1             ...                             0.0   
                  2             ...                             0.0   
                  3             ...                             0.0   
                  4             ...                             0.0   
                  5             ...                             0.0   

                      SNOW-PCTALL_GE001TI_FLAG  PRCP-PCTALL_GE100HI  \
STNID       MONTH DAY                                                 
AQC00914000 1     1                          P                 21.9   
                  2                          P                 21.9   
                  3                          P                 21.9   
                  4                          P                 21.9   
                  5                          P                 21.8   

                      PRCP-PCTALL_GE100HI_FLAG  PRCP-PCTALL_GE050HI  \
STNID       MONTH DAY                                                 
AQC00914000 1     1                          P                 38.2   
                  2                          P                 38.2   
                  3                          P                 38.2   
                  4                          P                 38.2   
                  5                          P                 38.2   

                      PRCP-PCTALL_GE050

In [10]:
import pyhs2

ImportError: No module named pyhs2

In [23]:
conn = pyhs2.connect(host='localhost', port=10000,authMechanism=None, user="hive", password=None,database='hive')

AttributeError: 'SQLContext' object has no attribute 'toDF'

In [ ]:
data.to_sql("weather_snow_rain", con=conn)